In [1]:
using ITensors
using ITensorMPS
using DataFrames
using CSV

# Define Parameters

In [36]:
# Site number N 
N = 80;

# Spin quantum number S 
S = 1/2;

# Max number of sweeps
max_sweep = 150;

# Max number of bond dimensions
D_max = 60;

# Max number of excited states calculated
max_states = 1;

# Penalty weight when calculating the excited states
weight = 100;

# DMRG algorithm parameters
BondDims = [10, 20, 40, D_max];
cutoff = [1e-10];
noise = [1e-7, 1e-8, 1e-10, 0, 1e-11, 1e-10, 1e-9, 1e-11, 0];

In [37]:
# Define operators for spin 3/2
ITensors.space(::SiteType"S=3/2") = 4

ITensors.op(::OpName"Sz",::SiteType"S=3/2") =
  [+3/2   0    0    0
     0  +1/2   0    0
     0    0  -1/2   0
     0    0    0  -3/2]

ITensors.op(::OpName"S+",::SiteType"S=3/2") =
  [0  √3  0  0
   0   0  2  0
   0   0  0 √3
   0   0  0  0]

ITensors.op(::OpName"S-",::SiteType"S=3/2") =
  [0   0  0   0
   √3  0  0   0
   0   2  0   0
   0   0  √3  0]

In [38]:
# Define the site space for spin 2
ITensors.space(::SiteType"S=2") = 5

# Define the Sz operator for spin 2
ITensors.op(::OpName"Sz", ::SiteType"S=2") = [
  2  0  0  0  0
  0  1  0  0  0
  0  0  0  0  0
  0  0  0 -1  0
  0  0  0  0 -2
]

# Define the S+ operator for spin 2
ITensors.op(::OpName"S+", ::SiteType"S=2") = [
  0  2    0     0     0
  0  0   √6     0     0
  0  0    0    √6     0
  0  0    0     0     2
  0  0    0     0     0
]

# Define the S- operator for spin 2
ITensors.op(::OpName"S-", ::SiteType"S=2") = [
  0     0     0     0  0
  2     0     0     0  0
  0    √6     0     0  0
  0     0    √6     0  0
  0     0     0     2  0
]


# Define Hamiltonians (Spin 1/2 and 1)

### AKLT Hamiltonian

In [39]:
function AKLT_Hamiltonian(N::Int64)

    os = OpSum() 
    for j=1:N-1 
        # S*S Terms
        # Sz Sz
        os += "Sz", j, "Sz", j+1

        # S+ S- and S- S+
        os += 1/2, "S+", j, "S-", j+1
        os += 1/2, "S-", j, "S+", j+1

        # Square terms in (S*S)^2
        # (1/3) * (Sz Sz)^2 term
        os += (1/3), "Sz", j, "Sz", j+1, "Sz", j, "Sz", j+1
        # (1/3) * (1/4) (S+ S- S+ S-) term
        os += (1/3) * (1/4), "S+", j, "S-", j+1, "S+", j, "S-", j+1
        os += (1/3) * (1/4), "S-", j, "S+", j+1, "S-", j, "S+", j+1

        # Cross terms in (S*S)^2
        # (1/3) * (1/2) Sz Sz S+ S- terms
        os += (1/3) * (1/2), "Sz", j, "Sz", j+1, "S+", j, "S-", j+1
        os += (1/3) * (1/2), "S+", j, "S-", j+1, "Sz", j, "Sz", j+1

        # (1/3) * (1/2) Sz Sz S- S+ terms
        os += (1/3) * (1/2), "Sz", j, "Sz", j+1, "S-", j, "S+", j+1
        os += (1/3) * (1/2), "S-", j, "S+", j+1, "Sz", j, "Sz", j+1

        # (1/3) * (1/4) S+ S- S- S+ terms
        os += (1/3) * (1/4), "S+", j, "S-", j+1, "S-", j, "S+", j+1
        os += (1/3) * (1/4), "S-", j, "S+", j+1, "S+", j, "S-", j+1
    end

    return os
end

AKLT_Hamiltonian (generic function with 1 method)

### Heisenberg Hamiltonian

In [40]:
function Heisenberg_Hamiltonian(N::Int64, J::Any)
    """
    This function is used to generate Hamiltonian with Opsum of the Heisenberg model
    N: the number of sites
    J: J coupling
    """
    os = OpSum()
    for n=1:N-1
        os += J, "Sz",n,"Sz",n+1
        os += J/2, "S+", n, "S-", n+1
        os += J/2, "S-", n, "S+", n+1
    end

    # periodic boundary conditions (PBCs)
    #os += J, "Sz",1,"Sz",N
    #os += J/2, "S+", 1, "S-", N
    #os += J/2, "S-", 1, "S+", N

    return os
end

Heisenberg_Hamiltonian (generic function with 1 method)

# AKLT Simulation

### DMRG Parameters

### Site and State Initialization

In [41]:
# Define site according to the spin type 
if S == 0.5
   sites = siteinds("S=1/2",N);
elseif S == 1
   sites = siteinds("S=1",N);
elseif S == 3/2
   sites = siteinds("S=3/2",N);
elseif S == 2
   sites = siteinds("S=2",N);
else
   println("Other spin model currently not available :(");
end

80-element Vector{Index{Int64}}:
 (dim=2|id=152|"S=1/2,Site,n=1")
 (dim=2|id=115|"S=1/2,Site,n=2")
 (dim=2|id=890|"S=1/2,Site,n=3")
 (dim=2|id=758|"S=1/2,Site,n=4")
 (dim=2|id=840|"S=1/2,Site,n=5")
 (dim=2|id=343|"S=1/2,Site,n=6")
 (dim=2|id=211|"S=1/2,Site,n=7")
 (dim=2|id=934|"S=1/2,Site,n=8")
 (dim=2|id=58|"S=1/2,Site,n=9")
 (dim=2|id=456|"S=1/2,Site,n=10")
 ⋮
 (dim=2|id=400|"S=1/2,Site,n=72")
 (dim=2|id=658|"S=1/2,Site,n=73")
 (dim=2|id=560|"S=1/2,Site,n=74")
 (dim=2|id=100|"S=1/2,Site,n=75")
 (dim=2|id=280|"S=1/2,Site,n=76")
 (dim=2|id=669|"S=1/2,Site,n=77")
 (dim=2|id=786|"S=1/2,Site,n=78")
 (dim=2|id=32|"S=1/2,Site,n=79")
 (dim=2|id=634|"S=1/2,Site,n=80")

In [42]:
function InitMPS(linkdim::Int, K::Int, WaveType::String, sites::Vector{Index{Int64}})
    # Create an array containing all the states we want to calculate 
    psi_arr = []
   
    if WaveType == "random"
       println("start with random states")
       for m = 1:K
           # Generate a random MPS
           psi_m = randomMPS(sites; linkdims = linkdim)
           # Add the new MPS to the array
           push!(psi_arr, psi_m)
       end
    else
       println("which initial states?!")
    end
    return psi_arr
end

InitMPS (generic function with 1 method)

In [43]:
# Create a random array of MPS using the function we just defined
psi_initials = InitMPS(10, max_states, "random", sites);

# Create a AKLT Hamiltonian MPO
os = Heisenberg_Hamiltonian(N,1);
H = MPO(os, sites);

# Define an observer to control the energy tolerance
obs = DMRGObserver(["Sz"], sites, energy_tol=1E-9);

start with random states


### Ground State Calculation

In [44]:
println("Start Calculating Groud State!")

# Use DMRG to calculate the ground state
temp_energy, temp_psi = dmrg(H, psi_initials[1], nsweeps=max_sweep, maxdim=BondDims, mindim=20, cutoff=cutoff, noise=noise, eigsolve_krylovdim=11, observer = obs);

# Calculate the norm of optimized ground state MPS
norm = inner(temp_psi, temp_psi);

# Calculate the normalized ground state energy
energy = temp_energy/norm;
println("The ground state energy is: ", energy)

Start Calculating Groud State!
After sweep 1 energy=-35.24443089392627  maxlinkdim=10 maxerr=1.68E-03 time=0.357
After sweep 2 energy=-35.26477018152224  maxlinkdim=20 maxerr=9.07E-07 time=0.531
After sweep 3 energy=-35.26523204423921  maxlinkdim=40 maxerr=3.43E-08 time=1.600
After sweep 4 energy=-35.26523701059902  maxlinkdim=80 maxerr=2.71E-10 time=4.012
After sweep 5 energy=-35.26523701339895  maxlinkdim=80 maxerr=3.57E-10 time=5.929
After sweep 6 energy=-35.2652370133923  maxlinkdim=80 maxerr=3.59E-10 time=5.815
Energy difference less than 1.0e-9, stopping DMRG
The ground state energy is: -35.26523701339225


### Ground State Correlation Matrix

In [45]:
# Calculate correlation matrix
Corr = correlation_matrix(temp_psi, "Sz", "Sz")

80×80 Matrix{Float64}:
  0.25         -0.217226      0.0665577    …   0.000558469  -0.000865527
 -0.217226      0.25         -0.0987777       -0.000360345   0.000558466
  0.0665577    -0.0987777     0.25             0.000676918  -0.0010491
 -0.0720227     0.05499      -0.187432        -0.000494906   0.000766999
  0.037155     -0.0308172     0.0627564        0.000765945  -0.00118706
 -0.0414573     0.0290498    -0.0654083    …  -0.000591704   0.000916997
  0.0252256    -0.0183636     0.035994         0.000841605  -0.0013043
 -0.0283545     0.019221     -0.0394918       -0.000671967   0.00104136
  0.0188216    -0.0130319     0.0251496        0.000910131  -0.00141047
 -0.0211826     0.0141249    -0.0279891       -0.000743358   0.00115195
  ⋮                                        ⋱                
 -0.00141044    0.000910105  -0.00170973      -0.0130319     0.0188216
  0.00104134   -0.000671952   0.00126229       0.019221     -0.0283545
 -0.00130427    0.00084158   -0.00158097      -0.018

In [46]:
# Convert the matrix to a DataFrame
df = DataFrame(Corr, :auto)

# Export the DataFrame to a CSV file
CSV.write("correlation_matrix.csv", df)

println("Correlation matrix has been exported to correlation_matrix.csv :3")

Correlation matrix has been exported to correlation_matrix.csv :3


### Entanglement Entropy